In [3]:
!cp -r "/content/drive/MyDrive/Professional Stuff/Technical Test/Verihubs" ./model

In [ ]:
!curl -L -o ./face-mask-detection.zip https://www.kaggle.com/api/v1/datasets/download/andrewmvd/face-mask-detection
!unzip -qq ./face-mask-detection.zip -d ./facemask_dataset

In [12]:
# !pip install --quiet rfdetr rfdetr[onnxexport]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 71.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.1/156.1 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 108.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.9/887.9 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 105.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.8/706.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━

In [1]:
from rfdetr import RFDETRBase

model = RFDETRBase(pretrain_weights="./model/checkpoint_best_total.pth")

Loading pretrain weights


## Conversion

To optimize the model, i choose to export the model to onnx and will run it later using onnxruntime because in my experience onnxruntime supports multiple "backends". We can run it with GPU and CPU, even on ARM CPU. Even optimized backends for certain hardware, for example intel's openVINO is also supported.

In [ ]:
# !pip install -U onnx==1.19 rfdetr[onnxexport]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 53.6 MB/s eta 0:00:00
  Attempting uninstall: onnx
    Found existing installation: onnx 1.20.1
    Uninstalling onnx-1.20.1:
      Successfully uninstalled onnx-1.20.1


In [2]:
model.export(simplify=True)

Exporting model to ONNX format


PyTorch inference output shapes - Boxes: torch.Size([1, 3900, 4]), Labels: torch.Size([1, 3900, 4])



Successfully exported ONNX model: output/inference_model.onnx
Successfully exported ONNX model to: output/inference_model.onnx
start simplify ONNX model: output/inference_model.onnx
[W] colored module is not installed, will not use colors when logging. To enable colors, please install the colored module: python3 -m pip install colored
[I] load onnx file: output/inference_model.onnx
[W] 'colored' module is not installed, will not use colors when logging. To enable colors, please install the 'colored' module: python3 -m pip install colored
[I] Folding Constants | Pass 1
[I]     Total Nodes | Original:  2008, After Folding:  1225 |   783 Nodes Folded
[I] Folding Constants | Pass 2
[I]     Total Nodes | Original:  1225, After Folding:  1026 |   199 Nodes Folded
[I] Folding Constants | Pass 3
[I]     Total Nodes | Original:  1026, After Folding:   945 |    81 Nodes Folded
[I] Folding Constants | Pass 4
[I]     Total Nodes | Original:   945, After Folding:   916 |    29 Nodes Folded
[I] Fol

In [9]:
!zip -r onnx_export.zip output/*

  adding: output/inference_model.onnx (deflated 14%)
  adding: output/inference_model.sim.onnx (deflated 11%)


In [10]:
!cp onnx_export.zip /content/drive/MyDrive/Professional\ Stuff/Technical\ Test/Verihubs/onnx_export.zip

After exporting, let's see how our training goes.

In [11]:
import json

with open("./model/results.json") as f:
    training_evals = json.load(f)

for classmap in training_evals["class_map"]["valid"]:
    for key, value in classmap.items():
        if key == "class":
            print(f"Evals for class {value}:")
        else:
            print(f" - {key}: {value}")


Evals for class with_mask:
 - map@50:95: 0.6387706161950641
 - map@50: 0.9496308582993108
 - precision: 0.9281842818428184
 - recall: 0.9332425068119891
 - f1_score: 0.9307065217391305
Evals for class without_mask:
 - map@50:95: 0.5061049133288853
 - map@50: 0.8529987209222274
 - precision: 0.9060773480662984
 - recall: 0.7488584474885844
 - f1_score: 0.82
Evals for class mask_weared_incorrect:
 - map@50:95: 0.3910092995812707
 - map@50: 0.5680709653479304
 - precision: 0.7619047619047619
 - recall: 0.48484848484848486
 - f1_score: 0.5925925925925926
Evals for class all:
 - map@50:95: 0.51196160970174
 - map@50: 0.7902335148564896
 - precision: 0.8653887972712928
 - recall: 0.7223164797163527
 - f1_score: 0.7810997047772409


As expected, the minority class suffers the most. Let's also try to see if it's also the case in the test set.

In [12]:
for classmap in training_evals["class_map"]["test"]:
    for key, value in classmap.items():
        if key == "class":
            print(f"Evals for class {value}:")
        else:
            print(f" - {key}: {value}")

Evals for class with_mask:
 - map@50:95: 0.6411089733597557
 - map@50: 0.9414547708314025
 - precision: 0.9156769596199525
 - recall: 0.8954703832752613
 - f1_score: 0.905460951262478
Evals for class without_mask:
 - map@50:95: 0.5197312392912448
 - map@50: 0.8597412628612452
 - precision: 0.8067226890756303
 - recall: 0.8067226890756303
 - f1_score: 0.8067226890756303
Evals for class mask_weared_incorrect:
 - map@50:95: 0.3655878718428366
 - map@50: 0.5475340345824206
 - precision: 0.7857142857142857
 - recall: 0.4782608695652174
 - f1_score: 0.5945945945945946
Evals for class all:
 - map@50:95: 0.5088093614979458
 - map@50: 0.782910022758356
 - precision: 0.8360379781366228
 - recall: 0.7268179806387031
 - f1_score: 0.768926078310901


Looking at the scores, it seems like the performence is close between eval and test, meaning the problem is not overfitting (variance) but is simply that we have too little data for the other classes. For the next runs, it is probably wiser to use per-class loss weighting.

### First analysis: box accuracy / precision

Overall, we can see that the map@50 is pretty far from the map@50:95. Meaning the model can somewhat reliably locate the faces, but it still struggles to put a tight fit around the object, either the box is not perfectly centered on the object or the box is too small / too big. TL;DR, its great at finding the general location, but pretty horrible at making sure the box is a tight fit, which may or may not be that big of a deal, depending on the use case. Again, the requirement doesnt really say anything so i can just analyze generally.

We can also see how much worse it is for the minority class (wrongmask). It follows a trend where the less images the class have, the lower it gets. We still see a healthy distance between the validation and test set performance though, so I'd say we're not suffering from overfitting, just need more data and variety for the minority class.

The cause can vary. It could be the images are too low quality, it could be we need better augmentations, it could be we havent trained long enough, it could also be that we need to rebalance the bbox loss vs classification loss during training. We will see later when we do visual inspection.

### Second analysis: precision-recall disparity

For the withmask and nomask classes, the precision and recall isnt too far apart. Meaning the false positives and false negatives are pretty balanced. 

Meanwhile for the wrongmask class, we can see the huge gap between the precision and recall. The recall is much lower, meaning the model is struggling with false negatives. Put simply, the model **FAILS** finding wrongmask faces. I can be like a broken radio and say we need more data, but on future runs, i think it is wise to apply a class weighting on our loss function so the model can pay more attention to the minority class. Oversampling the minority class might be another solution but it's probably redundant if we're using class weighting anyways. 

Interestingly enough, the nomask class has quite a disparity between validation and test

validation has precision in the 90s and recall 74, while recall is pretty balanced having both in the 80 ish range. I think despite our earlier effort, there is still quite a distribution difference that will need a better splitting method when we're creating our validation set. My hypothesis is that it's the images with a lot of nomask faces that tanks the validation recall.



Overall, most things are as expected, not many surprises here.